In [2]:
import sys
import os
from os import path
# 部署的路径
sys.path.append('/root/bdrisk/risk_project')
# 堡垒机的路径
sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))
# 这是log文件的存放路径
root_path = path.dirname(path.dirname(path.dirname(os.getcwd())))
from risk_models import *
# from risk_models import 
import datetime
import numpy as np

In [9]:
datetime.datetime.now().year

2021

In [2]:
# 企业货值与货类统计
sql='''
select c.UNF_SC_ID, a.TRADE_CODE,a.START_TIME,a.End_time,a.TRADE_NAME_CN,a.I_E_MARK,
a.CARGO_SUB_CATEGORY_CODE,a.CARGO_SUB_CATEGORY_NAME,a.SUM_GOODS_GROSS_VALUE_RMB
from DW_STA.FT_STA_GOODSOWNER_MAIN_CLASS a 
left join dw_corp_basic.dw_corp_cusdec c
on a.trade_code = c.CUS_CODE_ENT
inner join dim.DIM_FTZ_CORP b
on c.UNF_SC_ID = b.UNI_SC_ID
where c.isdeleted =0
'''

In [3]:
df_model1 = Read_Oracle().read_oracle(sql=sql, database='dbdw')

2021-06-16 17:49:18.831 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 2.741s


In [4]:
# 退单率查验率数据读取
sql='''select to_char(to_date(a.STARTDT),'yyyy/mm')  as startdt,
sum(SUM_ENTRY) as SUM_ENTRY,
sum(SUM_RETURN_FLAG) as SUM_RETURN_FLAG,
sum(SUM_INSPECTION) as SUM_INSPECTION,
sum(SUM_INSPECTION_NOPASS) as SUM_INSPECTION_NOPASS,
TRADE_CODE_SCC,
TRADE_NAME_CN 
from DW_STA.FT_CUS_DWS_ENTRY a 
left join dim.dim_trader c
on a.TRADE_CODE_KEY = c.TRADE_KEY
inner join dim.DIM_FTZ_CORP b
on c.TRADE_CODE_SCC = b.UNI_SC_ID
where c.iscurrent =1
group by TRADE_CODE_SCC,TRADE_NAME_CN, to_char(to_date(a.STARTDT),'yyyy/mm')'''

In [5]:
df_model2 = Read_Oracle().read_oracle(sql=sql, database='dbdw')

2021-06-16 17:49:29.011 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 10.12s


In [6]:
dt = datetime.datetime(2021, 5, 1)
target_dt = pd.DatetimeIndex([dt])[0]
org_code='9131000008793522XH'

In [7]:
df = df_model1.groupby(['UNF_SC_ID','START_TIME'])['SUM_GOODS_GROSS_VALUE_RMB'].sum().reset_index()
# 构建基础df
df = df[(df['START_TIME'] == target_dt)]

In [8]:
# 构建目标企业的df
final = df[(df['START_TIME'] == target_dt)& (df['UNF_SC_ID']==org_code)]
final

,UNF_SC_ID,START_TIME,SUM_GOODS_GROSS_VALUE_RMB
116,9131000008793522XH,2021-05-01,5.277566e+07


In [9]:
# 挖掘企业陌生货类
old_list = df_model1[df_model1['START_TIME'].apply(lambda x : x != target_dt) & (df_model1['UNF_SC_ID']==org_code)][['CARGO_SUB_CATEGORY_CODE','CARGO_SUB_CATEGORY_NAME']].to_dict('records')
new_list = df_model1[df_model1['START_TIME'].apply(lambda x : x == target_dt) & (df_model1['UNF_SC_ID']==org_code)][['CARGO_SUB_CATEGORY_CODE','CARGO_SUB_CATEGORY_NAME']].to_dict('records')
error_list=[]
for new_type in new_list:
    if new_type not in old_list:
        error_list.append(new_type)

In [10]:
# 列表去重，构建新的企业货类列表
new_l1 = []
for i in error_list:
    if i not in new_l1:
        new_l1.append(i)
new_l1

[{'CARGO_SUB_CATEGORY_CODE': '3901', 'CARGO_SUB_CATEGORY_NAME': '乙烯聚合物'},
 {'CARGO_SUB_CATEGORY_CODE': '7219', 'CARGO_SUB_CATEGORY_NAME': '不锈钢平板轧材'},
 {'CARGO_SUB_CATEGORY_CODE': '7210', 'CARGO_SUB_CATEGORY_NAME': '铁平板轧材'}]

In [29]:
final.loc[:,'NEW_HSCODE_LIST']= json.dumps(new_l1,ensure_ascii=False)

In [12]:
# 转化时间格式
df_model2['STARTDT'] = df_model2['STARTDT'].apply(lambda x: datetime.datetime.strptime(x, '%Y/%m'))
df_model2['RETURN_RATE'] = df_model2['SUM_INSPECTION_NOPASS'] / df_model2['SUM_ENTRY']
df_model2['RETURN_RATE']= df_model2['RETURN_RATE'].apply(lambda x : round(x,3))
df_model2['INSPECTION_RATE'] = df_model2['SUM_INSPECTION'] / df_model2['SUM_ENTRY']
df_model2['INSPECTION_RATE']= df_model2['INSPECTION_RATE'].apply(lambda x : round(x,3))
df2 = df_model2[['STARTDT','SUM_ENTRY','RETURN_RATE','INSPECTION_RATE','SUM_INSPECTION_NOPASS','TRADE_CODE_SCC','TRADE_NAME_CN']]

In [13]:
final = final.merge(df2,left_on=['UNF_SC_ID','START_TIME'],right_on=['TRADE_CODE_SCC','STARTDT'],how='left')
final = final.drop(columns=['STARTDT','TRADE_CODE_SCC'],axis=1)
final

,UNF_SC_ID,START_TIME,SUM_GOODS_GROSS_VALUE_RMB,NEW_HSCODE_LIST,SUM_ENTRY,RETURN_RATE,INSPECTION_RATE,SUM_INSPECTION_NOPASS,TRADE_NAME_CN
0,9131000008793522XH,2021-05-01,5.277566e+07,"[{""CARGO_SUB_CATEGORY_CODE"": ""3901"", ""CARGO_SU...",53,0.0,0.094,0,上海怡亚通供应链管理有限公司


In [14]:
final.rename(columns={'UNF_SC_ID':'ORG_CODE','START_TIME':'BIZ_DATE','SUM_GOODS_GROSS_VALUE_RMB':'ENTRY_VALUE','SUM_ENTRY':'ENTRY_NUM','SUM_INSPECTION_NOPASS':'NOPASS_NUM','TRADE_NAME_CN':'ORG_NAME'},inplace=True)

In [15]:
final

,ORG_CODE,BIZ_DATE,ENTRY_VALUE,NEW_HSCODE_LIST,ENTRY_NUM,RETURN_RATE,INSPECTION_RATE,NOPASS_NUM,ORG_NAME
0,9131000008793522XH,2021-05-01,5.277566e+07,"[{""CARGO_SUB_CATEGORY_CODE"": ""3901"", ""CARGO_SU...",53,0.0,0.094,0,上海怡亚通供应链管理有限公司


In [16]:
risk_a = final['NEW_HSCODE_LIST'].apply(lambda x: '发现{}起陌生交易货类异常'.format(len(json.loads(x))) if len(json.loads(x))> 0 else '')
risk_b = final['RETURN_RATE'].apply(lambda x: '退单率异常' if x >=0.1 else '')
risk_c = final['INSPECTION_RATE'].apply(lambda x: '查验率异常' if x >=0.1 else '')
risk_d = final['NOPASS_NUM'].apply(lambda x : '发现{}起海关查获事件'.format(x) if x > 0 else '')
final['RISK_LABEL'] = ';'.join(i for i in [risk_a[0],risk_b[0],risk_c[0],risk_d[0]] if i != '')

In [17]:
score_a = final['NEW_HSCODE_LIST'].apply(lambda x: -0.2 * len(json.loads(x)) if len(json.loads(x))> 0 else 0)
score_b = final['RETURN_RATE'].apply(lambda x: -0.1 * ((x-0.1)/0.1)if x >=0.1 else 0)
score_c = final['INSPECTION_RATE'].apply(lambda x: -0.1 * ((x-0.1)/0.1)if x >=0.1 else 0)
score_d = final['NOPASS_NUM'].apply(lambda x : -1 * x if x > 0 else 0)
final['SCORE'] = score_a + score_b + score_c +score_d

In [18]:
final['CHECK_TIME'] = datetime.datetime.strftime(datetime.datetime.now(), format='%Y-%m-%d %H:%M:%S')
final['CHECK_TIME'] = pd.to_datetime(final['CHECK_TIME'], format='%Y-%m-%d %H:%M:%S')

In [19]:
final = final.reset_index().rename(columns={'index': 'ID'})

In [26]:
Write_Oracle().write_oracle('BD_RISK_RESULT_TRADE_TD2', final, org_code=org_code, alarm=None, iscurrent=False)

2021-06-16 17:50:24.433 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.082s
2021-06-16 17:50:24.543 | INFO     | risk_models.config.write_config.write_func:write_oracle:181 - Processing... Writing 1 rows into database
2021-06-16 17:50:24.582 | INFO     | risk_models.config.write_config.write_func:write_oracle:188 - Insert data into BD_RISK_RESULT_TRADE_TD2 successfully! Total write time spent 0.237s


In [27]:
alarm_reason = []
alarm_reason = [i for i in [risk_a[0],risk_b[0],risk_c[0],risk_d[0]] if i != '']
df_alarm = []
for i in alarm_reason:
    temp = {}
    temp['CHECK_TIME'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    temp['ORG_CODE'] = org_code
    temp['MODEL_CODE'] = 'TRADE'
    temp['CHILD_MODEL_CODE'] = 'TR2'
    temp['ALARM_REASON'] = i
    temp['ALARM_NUMBER'] = int(re.findall('发现(.*?)起.*?', i)[0]) +len(re.findall('退单率',i)) +len(re.findall('查验率',i)) 
    df_alarm.append(temp)
df_alarm = pd.DataFrame(df_alarm)
df_alarm = df_alarm.reset_index().rename(columns={'index': 'ID'})
df_alarm['CHECK_TIME'] = pd.to_datetime(df_alarm['CHECK_TIME'], format='%Y-%m-%d %H:%M:%S')
if df_alarm.empty:
    print('没有异常情况')
else:
    Write_Oracle().write_oracle('BD_RISK_ALARM_ITEM', df_alarm, org_code=self.org_code,
                                alarm=[self.model_code, self.child_model_code])

NameError: name 'self' is not defined

In [22]:
df_alarm

,ID,CHECK_TIME,ORG_CODE,MODEL_CODE,CHILD_MODEL_CODE,ALARM_REASON,ALARM_NUMBER
0,0,2021-06-16 17:49:30,9131000008793522XH,model_code,child_model_code,发现3起陌生交易货类异常,3
